# 🧠 MedPrintAI — Treinamento nnU-Net

**Pré-requisitos:**
1. Runtime → Alterar tipo de runtime → **T4 GPU**
2. Ter o dataset convertido (`Dataset001_BoneLesions/`) no Google Drive

**Estrutura esperada no Drive:**
```
MedPrintAI/
└── Dataset001_BoneLesions/
    ├── dataset.json
    ├── imagesTr/
    ├── labelsTr/
    └── imagesTs/
```

## 1. Verificar GPU

In [ ]:
import torch
print(f'GPU disponível: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
else:
    print('⚠️  Sem GPU! Vá em Runtime → Alterar tipo de runtime → T4 GPU')

## 2. Instalar nnU-Net

In [ ]:
%%capture
!pip install nnunetv2

## 3. Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Ajuste este caminho se necessário
DRIVE_DATASET = '/content/drive/MyDrive/MedPrintAI/Dataset001_BoneLesions'

import os
assert os.path.exists(DRIVE_DATASET), f'Dataset não encontrado em {DRIVE_DATASET}'
print(f'✅ Dataset encontrado: {DRIVE_DATASET}')

## 4. Configurar variáveis de ambiente do nnU-Net

In [ ]:
import os

# Pasta raiz onde ficam os datasets raw (deve conter Dataset001_BoneLesions/)
NNUNET_RAW          = '/content/nnunet_raw'
NNUNET_PREPROCESSED = '/content/nnunet_preprocessed'
NNUNET_RESULTS      = '/content/nnunet_results'

os.makedirs(NNUNET_RAW, exist_ok=True)
os.makedirs(NNUNET_PREPROCESSED, exist_ok=True)
os.makedirs(NNUNET_RESULTS, exist_ok=True)

# Cria symlink do Drive → pasta local
dst = os.path.join(NNUNET_RAW, 'Dataset001_BoneLesions')
if not os.path.exists(dst):
    os.symlink(DRIVE_DATASET, dst)

os.environ['nnUNet_raw']          = NNUNET_RAW
os.environ['nnUNet_preprocessed'] = NNUNET_PREPROCESSED
os.environ['nnUNet_results']      = NNUNET_RESULTS

print('✅ Variáveis de ambiente configuradas')
print(f'  nnUNet_raw          = {NNUNET_RAW}')
print(f'  nnUNet_preprocessed = {NNUNET_PREPROCESSED}')
print(f'  nnUNet_results      = {NNUNET_RESULTS}')

## 5. Validar e pré-processar o dataset

Este passo analisa o dataset e configura automaticamente a arquitetura da rede (resolução, tamanho de batch, etc.).

In [ ]:
!nnUNetv2_plan_and_preprocess -d 001 --verify_dataset_integrity -np 2

## 6. Treinar (5-fold cross-validation)

Cada fold leva ~30-60 minutos com T4. Para ter o modelo completo você precisa rodar os 5 folds.  
Se tiver pressa, rode só o fold 0 — já dá um modelo funcional.

In [ ]:
# Treina fold 0 (rode mais vezes mudando o último argumento: 1, 2, 3, 4)
!nnUNetv2_train 001 3d_fullres 0 --npz

In [ ]:
# Opcional: treinar todos os folds de uma vez (mais longo)
# for fold in range(5):
#     !nnUNetv2_train 001 3d_fullres {fold} --npz

## 7. Avaliação (métricas)

Após treinar todos os folds, calcula o Dice Score médio.

In [ ]:
!nnUNetv2_find_best_configuration 001 -c 3d_fullres

## 8. Inferência de teste (opcional)

Roda o modelo nos CTs em `imagesTs/` (sem ground truth).

In [ ]:
PRED_DIR = '/content/predicoes_teste'
os.makedirs(PRED_DIR, exist_ok=True)

!nnUNetv2_predict \
  -i {NNUNET_RAW}/Dataset001_BoneLesions/imagesTs \
  -o {PRED_DIR} \
  -d 001 \
  -c 3d_fullres \
  -f 0

## 9. Salvar modelo no Google Drive

Copia os resultados para o Drive para não perder quando a sessão encerrar.

In [ ]:
import shutil

DRIVE_RESULTADOS = '/content/drive/MyDrive/MedPrintAI/modelo_treinado'
os.makedirs(DRIVE_RESULTADOS, exist_ok=True)

shutil.copytree(NNUNET_RESULTS, DRIVE_RESULTADOS, dirs_exist_ok=True)
print(f'✅ Modelo salvo em: {DRIVE_RESULTADOS}')
print()
print('Para usar localmente:')
print('  1. Baixe a pasta modelo_treinado/ do Drive')
print('  2. Coloque em: modelo_ia/modelo_treinado/')
print('  3. Rode: python treinamento/inferencia.py --input seu_ct.nii.gz')